# Get the greenspaces from Los Angeles and set the flags

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from glob2 import glob
import osmnx as ox

In [2]:
green=gpd.read_file('../data/outputs/greenspaces_la_final.shp')

In [3]:
green.shape

(12947, 2)

In [4]:
green.head()

,FID,geometry
0,0,"MULTIPOLYGON (((-117.75412 33.52781, -117.7540..."
1,1,"POLYGON ((-117.64313 33.64055, -117.64253 33.6..."
2,2,"POLYGON ((-117.72871 33.64874, -117.72877 33.6..."
3,3,"POLYGON ((-117.73685 33.71229, -117.73677 33.7..."
4,4,"POLYGON ((-117.74009 33.79436, -117.74004 33.7..."


In [5]:
#get California as gdf to check anomalies later. Points outside California will be excluded
ca=ox.geocode_to_gdf('California')

In [6]:
ca

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"MULTIPOLYGON (((-124.48200 40.44032, -124.4813...",42.009499,32.529524,-114.130782,-124.482003,259044285,relation,165475,36.701463,-118.755997,"California, United States",boundary,administrative,0.922136


## Load observations (API)

In [7]:
data_folder = '../data/observations_final'
df = pd.concat([pd.read_csv(f).assign(challenge=f.replace('.csv','')) for f in glob(data_folder+'/CNC_Los_Angeles_*.csv')])

In [8]:
df.shape

(91694, 39)

In [9]:
#check for incorrect points (lat)
df['latitude'].min(), df['latitude'].max()

(32.792719780300004, 76.3489192018)

In [10]:
#check for incorrect points (lon). Clearly something wrong!
df['longitude'].min(), df['longitude'].max()

(-118.9965502909, 130.01725921940002)

## Get only the points inside CA

In [11]:
df=df[df['latitude'].le(ca['bbox_north'].iloc[0]) \
                  & df['latitude'].ge(ca['bbox_south'].iloc[0]) \
                  & df['longitude'].le(ca['bbox_east'].iloc[0]) \
                  & df['longitude'].ge(ca['bbox_west'].iloc[0])]

In [12]:
df.shape

(91693, 39)

In [13]:
df.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,iconic_taxon_name,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge
0,5591391,Fri Apr 14 2017 09:34:12 GMT-0700 (PDT),2017-04-14T09:34:12-07:00,America/Los_Angeles,2017-04-06T09:34:29-07:00,2018-01-13T14:28:01-08:00,NaN,436798,laurachun,research,...,Plantae,species,52855,True,False,False,11983,11983,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017
1,5591694,Sat Apr 15 2017 10:18:50 GMT-0700 (PDT),2017-04-15T10:18:50-07:00,America/Los_Angeles,2017-04-06T10:19:03-07:00,2018-01-13T14:28:03-08:00,NaN,436798,laurachun,research,...,Plantae,species,53145,True,False,False,6476,6476,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017
2,5684540,Fri Apr 14 2017 00:03:16 GMT-0700 (PDT),2017-04-14T00:03:16-07:00,America/Los_Angeles,2017-04-14T00:04:16-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,...,Insecta,family,52380,False,False,False,55396,55396,"[{'user_id': 275891, 'category': 'supporting',...",../data/observations_final\CNC_Los_Angeles_2017
3,5684550,Fri Apr 14 2017 00:04:42 GMT-0700 (PDT),2017-04-14T00:04:42-07:00,America/Los_Angeles,2017-04-14T00:05:31-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,...,Mollusca,genus,933174,False,False,False,8938,8938,"[{'user_id': 81261, 'category': 'improving', '...",../data/observations_final\CNC_Los_Angeles_2017
4,5684551,Fri Apr 14 2017 00:04:06 GMT-0700 (PDT),2017-04-14T00:04:06-07:00,America/Los_Angeles,2017-04-14T00:05:32-07:00,2017-04-14T05:30:19-07:00,NaN,15419,mordenana,needs_id,...,Arachnida,order,47119,False,False,False,1420015,1420015,"[{'user_id': 15419, 'category': None, 'disagre...",../data/observations_final\CNC_Los_Angeles_2017


## Create gdf using the observations

In [14]:
df['longitude'].isna().sum(), df['latitude'].isna().sum()

(0, 0)

In [15]:
#create the points inverting the coordinates
geometry_invert = [Point(xy) for xy in zip(df.longitude, df.latitude)]

In [16]:
#crete a geodataframe with the observations
observations_gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry=geometry_invert)

In [17]:
observations_gdf.shape

(91693, 40)

In [18]:
observations_gdf.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry
0,5591391,Fri Apr 14 2017 09:34:12 GMT-0700 (PDT),2017-04-14T09:34:12-07:00,America/Los_Angeles,2017-04-06T09:34:29-07:00,2018-01-13T14:28:01-08:00,NaN,436798,laurachun,research,...,species,52855,True,False,False,11983,11983,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20138 34.09752)
1,5591694,Sat Apr 15 2017 10:18:50 GMT-0700 (PDT),2017-04-15T10:18:50-07:00,America/Los_Angeles,2017-04-06T10:19:03-07:00,2018-01-13T14:28:03-08:00,NaN,436798,laurachun,research,...,species,53145,True,False,False,6476,6476,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20141 34.09936)
2,5684540,Fri Apr 14 2017 00:03:16 GMT-0700 (PDT),2017-04-14T00:03:16-07:00,America/Los_Angeles,2017-04-14T00:04:16-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,...,family,52380,False,False,False,55396,55396,"[{'user_id': 275891, 'category': 'supporting',...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.31088 34.07109)
3,5684550,Fri Apr 14 2017 00:04:42 GMT-0700 (PDT),2017-04-14T00:04:42-07:00,America/Los_Angeles,2017-04-14T00:05:31-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,...,genus,933174,False,False,False,8938,8938,"[{'user_id': 81261, 'category': 'improving', '...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.31077 34.07111)
4,5684551,Fri Apr 14 2017 00:04:06 GMT-0700 (PDT),2017-04-14T00:04:06-07:00,America/Los_Angeles,2017-04-14T00:05:32-07:00,2017-04-14T05:30:19-07:00,NaN,15419,mordenana,needs_id,...,order,47119,False,False,False,1420015,1420015,"[{'user_id': 15419, 'category': None, 'disagre...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-117.99642 34.14409)


## Spatial join

In [19]:
# spatial join observations and observations.
points_polys = gpd.sjoin(observations_gdf, green, how='inner')

In [20]:
points_polys.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry,index_right,FID
0,5591391,Fri Apr 14 2017 09:34:12 GMT-0700 (PDT),2017-04-14T09:34:12-07:00,America/Los_Angeles,2017-04-06T09:34:29-07:00,2018-01-13T14:28:01-08:00,NaN,436798,laurachun,research,...,True,False,False,11983,11983,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20138 34.09752),8651,8651
1,5591694,Sat Apr 15 2017 10:18:50 GMT-0700 (PDT),2017-04-15T10:18:50-07:00,America/Los_Angeles,2017-04-06T10:19:03-07:00,2018-01-13T14:28:03-08:00,NaN,436798,laurachun,research,...,True,False,False,6476,6476,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20141 34.09936),8651,8651
3697,5733872,Sat Apr 15 2017 14:09:39 GMT-0700 (PDT),2017-04-15T14:09:39-07:00,America/Los_Angeles,2017-04-15T14:13:45-07:00,2018-01-13T14:44:03-08:00,"Observation and photo by Martha Singer, sent t...",262752,alexbroad,research,...,True,False,False,17659,17659,"[{'user_id': 262752, 'category': 'improving', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20125 34.09777),8651,8651
3702,5733938,Sat Apr 15 2017 07:54:21 GMT-0700 (PDT),2017-04-15T07:54:21-07:00,America/Los_Angeles,2017-04-15T14:14:33-07:00,2018-01-13T14:44:04-08:00,"Observation and photo by Martha Singer, sent t...",262752,alexbroad,needs_id,...,True,False,False,105228,105228,"[{'user_id': 262752, 'category': None, 'disagr...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20125 34.09777),8651,8651
3707,5734002,Sat Apr 15 2017 10:05:04 GMT-0700 (PDT),2017-04-15T10:05:04-07:00,America/Los_Angeles,2017-04-15T14:16:11-07:00,2018-01-13T14:44:05-08:00,"Observation and photo by Martha Singer, sent t...",262752,alexbroad,research,...,True,False,False,3866,3866,"[{'user_id': 262752, 'category': 'improving', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20125 34.09777),8651,8651


In [21]:
points_polys.shape

(39028, 42)

In [22]:
points_polys.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Set flag for greenspaces

In [23]:
ids_with_greenspace = points_polys.id.to_list()

In [24]:
new_df = df.set_index('id')
new_df.head()

,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,reviewed_by,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry
id,,,,,,,,,,,,,,,,,,,,,
5591391,Fri Apr 14 2017 09:34:12 GMT-0700 (PDT),2017-04-14T09:34:12-07:00,America/Los_Angeles,2017-04-06T09:34:29-07:00,2018-01-13T14:28:01-08:00,NaN,436798,laurachun,research,"[1850, 73705, 436798]",...,species,52855,True,False,False,11983,11983,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20138 34.09752)
5591694,Sat Apr 15 2017 10:18:50 GMT-0700 (PDT),2017-04-15T10:18:50-07:00,America/Los_Angeles,2017-04-06T10:19:03-07:00,2018-01-13T14:28:03-08:00,NaN,436798,laurachun,research,"[1850, 6971, 73705, 436798]",...,species,53145,True,False,False,6476,6476,"[{'user_id': 73705, 'category': 'supporting', ...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.20141 34.09936)
5684540,Fri Apr 14 2017 00:03:16 GMT-0700 (PDT),2017-04-14T00:03:16-07:00,America/Los_Angeles,2017-04-14T00:04:16-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,"[1850, 6743, 52839, 275891]",...,family,52380,False,False,False,55396,55396,"[{'user_id': 275891, 'category': 'supporting',...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.31088 34.07109)
5684550,Fri Apr 14 2017 00:04:42 GMT-0700 (PDT),2017-04-14T00:04:42-07:00,America/Los_Angeles,2017-04-14T00:05:31-07:00,2018-01-13T14:31:44-08:00,NaN,6743,lhiggins,needs_id,"[1850, 6743, 10285, 12809, 73705, 81261, 440756]",...,genus,933174,False,False,False,8938,8938,"[{'user_id': 81261, 'category': 'improving', '...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-118.31077 34.07111)
5684551,Fri Apr 14 2017 00:04:06 GMT-0700 (PDT),2017-04-14T00:04:06-07:00,America/Los_Angeles,2017-04-14T00:05:32-07:00,2017-04-14T05:30:19-07:00,NaN,15419,mordenana,needs_id,"[1850, 15419]",...,order,47119,False,False,False,1420015,1420015,"[{'user_id': 15419, 'category': None, 'disagre...",../data/observations_final\CNC_Los_Angeles_2017,POINT (-117.99642 34.14409)


In [25]:
new_df['greenspace_flag'] = '0'
new_df.loc[ids_with_greenspace,'greenspace_flag'] = '1'

In [26]:
new_df.greenspace_flag.value_counts()

0    52665
1    39028
Name: greenspace_flag, dtype: int64

In [27]:
#export as csv
new_df.to_csv('../data/outputs/la_data_with_greenspace_flag_v2.csv')